# Rayleigh distribution (`rayleigh`)

The **Rayleigh distribution** is a **continuous** distribution on nonnegative values. It appears whenever you take the **magnitude of a 2D isotropic Gaussian vector**:

\[
X, Y \overset{i.i.d.}{\sim} \mathcal{N}(0,\sigma^2) \quad\Longrightarrow\quad R=\sqrt{X^2+Y^2} \sim \text{Rayleigh}(\sigma).
\]

This makes it a natural model for **amplitudes** (radar/sonar returns, wireless fading, I/Q noise magnitude), **radial distances** in the plane, and any strictly nonnegative measurement driven by two independent Gaussian components.

## What you’ll learn
- how to recognize Rayleigh-shaped data and how it relates to Gaussians, Weibull, Exponential, and Rice distributions
- the PDF/CDF (and quantiles) in LaTeX
- key moments (mean/variance/skewness/kurtosis), MGF/characteristic function, and entropy
- parameter interpretation (how changing \(\sigma\) changes the shape)
- core derivations: \(\mathbb{E}[X]\), \(\mathrm{Var}(X)\), and the likelihood + MLE
- NumPy-only sampling via inverse transform + Monte Carlo validation
- SciPy usage: `scipy.stats.rayleigh` (`pdf`, `cdf`, `rvs`, `fit`)
- practical statistical use cases (testing, Bayesian updates, generative modeling)


## Notebook roadmap
1) Title & classification
2) Intuition & motivation
3) Formal definition (PDF/CDF)
4) Moments & properties
5) Parameter interpretation
6) Derivations (expectation, variance, likelihood)
7) Sampling & simulation (NumPy-only)
8) Visualization (PDF, CDF, Monte Carlo)
9) SciPy integration (`scipy.stats.rayleigh`)
10) Statistical use cases
11) Pitfalls
12) Summary


## Prerequisites & notation
We assume comfort with:
- basic calculus (change of variables in integrals)
- expectations/variance and likelihood
- standard special functions: \(\Gamma(\cdot)\) and the error function \(\operatorname{erf}(\cdot)\)

**Parameterization used here**
- Canonical Rayleigh uses a single **scale** parameter \(\sigma>0\) with support \(x\ge 0\).
- SciPy uses `stats.rayleigh(loc, scale)`. In this notebook we mostly take `loc=0` and identify \(\sigma = \texttt{scale}\).

Throughout, let \(X\sim\text{Rayleigh}(\sigma)\) denote the canonical (\(\texttt{loc}=0\)) distribution.


In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import plotly.io as pio

from scipy import stats
from scipy.constants import euler_gamma
from scipy.special import erf

pio.templates.default = "plotly_white"
pio.renderers.default = os.environ.get("PLOTLY_RENDERER", "notebook")
np.set_printoptions(precision=4, suppress=True)

rng = np.random.default_rng(42)


## 1) Title & classification

| Item | Value |
|---|---|
| Name | Rayleigh (`rayleigh`) |
| Type | Continuous |
| Support | \(x\in[0,\infty)\) |
| Parameter space | scale \(\sigma>0\) |
| SciPy form | `stats.rayleigh(loc, scale)` with support \(x\ge \texttt{loc}\) |

In this notebook we focus on the **canonical** form with `loc=0`.


## 2) Intuition & motivation

### What it models
Think of a point \((X,Y)\) in the plane whose coordinates are independent Gaussian noise with the same variance:

\[
X,Y\overset{i.i.d.}{\sim}\mathcal{N}(0,\sigma^2).
\]

The radius \(R=\sqrt{X^2+Y^2}\) is always nonnegative. Because the Gaussian is **rotationally symmetric**, angle is uniform and all the interesting mass is in the radial direction — that radial distribution is Rayleigh.

A quick derivation sketch (polar coordinates): if
\(f_{X,Y}(x,y)=\frac{1}{2\pi\sigma^2}\exp\{-\tfrac{x^2+y^2}{2\sigma^2}\}\), then with \(x=r\cos\theta\), \(y=r\sin\theta\) and Jacobian \(|J|=r\),

\[
 f_{R,\Theta}(r,\theta)=\frac{1}{2\pi\sigma^2}\exp\{-\tfrac{r^2}{2\sigma^2}\}\,r.
\]

Integrating out \(\theta\in[0,2\pi)\) yields \(f_R(r)=\frac{r}{\sigma^2}e^{-r^2/(2\sigma^2)}\) for \(r\ge 0\).

### Typical real-world use cases
- **Wireless communications (Rayleigh fading)**: received signal amplitude under many small multipath reflections (no dominant line-of-sight component).
- **Radar/sonar envelope detection**: magnitude of complex Gaussian noise after I/Q demodulation.
- **MRI / complex-valued imaging**: magnitude of complex Gaussian noise (Rayleigh is the noise floor when the true signal is near zero).
- **Wind speed**: often Weibull is used; Rayleigh is a common *special case* (shape fixed at 2).

### Relations to other distributions
- **Chi distribution**: Rayleigh is \(\chi\) with \(k=2\) degrees of freedom.
- **Weibull**: Rayleigh is Weibull with shape \(k=2\) and scale \(\lambda=\sigma\sqrt{2}\).
- **Exponential (via squaring)**: \(X^2\) is exponential:
  \[
  X\sim\text{Rayleigh}(\sigma)\quad\Longrightarrow\quad X^2\sim\text{Exp}\Big(\text{rate}=\frac{1}{2\sigma^2}\Big).
  \]
- **Rice (Rician)**: magnitude of a 2D Gaussian with *nonzero mean*; Rayleigh is the special case with mean vector \(0\).


## 3) Formal definition

Let \(X\sim\text{Rayleigh}(\sigma)\) with \(\sigma>0\).

### PDF
\[
 f(x;\sigma)=\begin{cases}
\frac{x}{\sigma^2}\exp\left(-\frac{x^2}{2\sigma^2}\right), & x\ge 0\\
0, & x<0.
\end{cases}
\]

### CDF
\[
 F(x;\sigma)=\mathbb{P}(X\le x)=\begin{cases}
1-\exp\left(-\frac{x^2}{2\sigma^2}\right), & x\ge 0\\
0, & x<0.
\end{cases}
\]

### Quantiles (inverse CDF)
For \(u\in(0,1)\),
\[
F^{-1}(u)=\sigma\sqrt{-2\ln(1-u)}.
\]

A useful byproduct is the **survival** \(S(x)=1-F(x)=\exp\{-x^2/(2\sigma^2)\}\) and the **hazard rate**
\(h(x)=f(x)/S(x)=x/\sigma^2\), which increases linearly.


In [ ]:
def rayleigh_pdf(x, sigma):
    """PDF of Rayleigh(sigma), support x >= 0."""
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    out = np.zeros_like(x, dtype=float)
    mask = x >= 0
    xm = x[mask]
    out[mask] = (xm / sigma**2) * np.exp(-(xm**2) / (2 * sigma**2))
    return out


def rayleigh_cdf(x, sigma):
    """CDF of Rayleigh(sigma), stable near 0 via expm1."""
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    out = np.zeros_like(x, dtype=float)
    mask = x >= 0
    xm = x[mask]
    out[mask] = -np.expm1(-(xm**2) / (2 * sigma**2))
    return out


def rayleigh_logpdf(x, sigma):
    """Log-PDF of Rayleigh(sigma). Returns -inf for x <= 0."""
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    out = np.full_like(x, -np.inf, dtype=float)
    mask = x > 0
    xm = x[mask]
    out[mask] = np.log(xm) - 2 * np.log(sigma) - (xm**2) / (2 * sigma**2)
    return out


def rayleigh_ppf(u, sigma):
    """Inverse CDF (percent point function), for u in [0, 1)."""
    u = np.asarray(u, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    if np.any((u < 0) | (u >= 1)):
        raise ValueError("u must be in [0, 1)")

    return sigma * np.sqrt(-2.0 * np.log1p(-u))


## 4) Moments & properties

For \(X\sim\text{Rayleigh}(\sigma)\):

| Quantity | Value |
|---|---|
| Mean | \(\mathbb{E}[X]=\sigma\sqrt{\pi/2}\) |
| Variance | \(\mathrm{Var}(X)=\frac{4-\pi}{2}\,\sigma^2\) |
| Skewness | \(\gamma_1=\frac{2\sqrt{\pi}(\pi-3)}{(4-\pi)^{3/2}}\) |
| Excess kurtosis | \(\gamma_2=\frac{-6\pi^2+24\pi-16}{(4-\pi)^2}\) (so kurtosis \(=3+\gamma_2\)) |
| Mode | \(\sigma\) |
| Median | \(\sigma\sqrt{2\ln 2}\) |
| \(k\)-th moment | \(\mathbb{E}[X^k]=\sigma^k 2^{k/2}\,\Gamma(1+k/2)\) |
| MGF | \(M_X(t)=1+\sigma t\sqrt{\pi/2}\,e^{\sigma^2 t^2/2}\big(1+\mathrm{erf}(\sigma t/\sqrt{2})\big)\) |
| CF | \(\varphi_X(\omega)=M_X(i\omega)\) |
| Entropy | \(H=1+\ln(\sigma/\sqrt{2})+\tfrac{\gamma}{2}\) (nats; \(\gamma\) is Euler–Mascheroni) |

Notable properties:
- **Scaling**: if \(X\sim\text{Rayleigh}(\sigma)\) and \(c>0\), then \(cX\sim\text{Rayleigh}(c\sigma)\).
- **Squaring**: \(X^2\) is exponential: \(X^2\sim\text{Exp}(\text{rate}=1/(2\sigma^2))\).
- **Increasing hazard**: \(h(x)=x/\sigma^2\) grows with \(x\) (useful in reliability modeling).


In [ ]:
def rayleigh_mean(sigma):
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    return sigma * np.sqrt(np.pi / 2)


def rayleigh_var(sigma):
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    return ((4 - np.pi) / 2) * sigma**2


def rayleigh_skewness():
    return (2 * np.sqrt(np.pi) * (np.pi - 3)) / (4 - np.pi) ** (3 / 2)


def rayleigh_excess_kurtosis():
    return (-6 * np.pi**2 + 24 * np.pi - 16) / (4 - np.pi) ** 2


def rayleigh_entropy(sigma):
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    return 1 + np.log(sigma / np.sqrt(2)) + 0.5 * euler_gamma


def rayleigh_mgf(t, sigma):
    """MGF M(t) = E[exp(tX)] for X ~ Rayleigh(sigma). Works for real or complex t."""
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    t = np.asarray(t)
    z = sigma * t / np.sqrt(2)
    return 1 + sigma * t * np.sqrt(np.pi / 2) * np.exp(0.5 * (sigma * t) ** 2) * (1 + erf(z))


# Quick numeric check against Monte Carlo
sigma = 1.7
n = 200_000
samples = rayleigh_ppf(rng.random(n), sigma=sigma)

print("Mean (theory, MC):", rayleigh_mean(sigma), samples.mean())
print("Var  (theory, MC):", rayleigh_var(sigma), samples.var())
print("Skewness (theory):", rayleigh_skewness())
print("Excess kurtosis (theory):", rayleigh_excess_kurtosis())
print("Entropy (theory):", rayleigh_entropy(sigma))

# MGF check at a couple of points
for t in [0.2, 0.8]:
    mc = np.mean(np.exp(t * samples))
    th = rayleigh_mgf(t, sigma)
    print(f"MGF at t={t}: theory={th:.6f}, MC={mc:.6f}")


## 5) Parameter interpretation

The Rayleigh distribution has a **single scale parameter** \(\sigma\): it stretches the distribution horizontally.

- If \(X\sim\text{Rayleigh}(\sigma)\), then \(X/\sigma\sim\text{Rayleigh}(1)\).
- Typical size grows linearly with \(\sigma\): \(\mathbb{E}[X]=\sigma\sqrt{\pi/2}\).
- Dispersion grows quadratically with \(\sigma\): \(\mathrm{Var}(X)\propto\sigma^2\).

Shape landmarks:
- **Mode**: \(x_{\text{mode}}=\sigma\)
- **Median**: \(x_{0.5}=\sigma\sqrt{2\ln 2}\)
- **\(p\)-quantile**: \(x_p=\sigma\sqrt{-2\ln(1-p)}\)

Below, we visualize how changing \(\sigma\) changes the PDF and CDF.


In [ ]:
sigmas = [0.5, 1.0, 2.0]

x = np.linspace(0, 10, 600)

# PDF
fig = go.Figure()
for s in sigmas:
    fig.add_trace(go.Scatter(x=x, y=rayleigh_pdf(x, s), mode="lines", name=f"σ={s:g}"))

fig.update_layout(
    title="Rayleigh PDF: effect of the scale σ",
    xaxis_title="x",
    yaxis_title="pdf",
)
fig.show()

# CDF
fig = go.Figure()
for s in sigmas:
    fig.add_trace(go.Scatter(x=x, y=rayleigh_cdf(x, s), mode="lines", name=f"σ={s:g}"))

fig.update_layout(
    title="Rayleigh CDF: effect of the scale σ",
    xaxis_title="x",
    yaxis_title="cdf",
)
fig.show()


## 6) Derivations

### Expectation
Start from the PDF and compute
\[
\mathbb{E}[X]=\int_0^\infty x\,\frac{x}{\sigma^2}e^{-x^2/(2\sigma^2)}\,dx
=\frac{1}{\sigma^2}\int_0^\infty x^2 e^{-x^2/(2\sigma^2)}\,dx.
\]

Use the substitution \(u=\frac{x^2}{2\sigma^2}\) (so \(x=\sigma\sqrt{2u}\), \(dx=\sigma\frac{1}{\sqrt{2u}}du\)):
\[
\mathbb{E}[X]=\sigma\sqrt{2}\int_0^\infty u^{1/2}e^{-u}\,du
=\sigma\sqrt{2}\,\Gamma\left(\tfrac{3}{2}\right)
=\sigma\sqrt{\tfrac{\pi}{2}}.
\]

### Variance
Similarly,
\[
\mathbb{E}[X^2]=\int_0^\infty x^2\,\frac{x}{\sigma^2}e^{-x^2/(2\sigma^2)}\,dx
=\frac{1}{\sigma^2}\int_0^\infty x^3 e^{-x^2/(2\sigma^2)}\,dx.
\]

With the same substitution \(u=x^2/(2\sigma^2)\), this becomes
\(
\mathbb{E}[X^2]=2\sigma^2\int_0^\infty u e^{-u}\,du=2\sigma^2\Gamma(2)=2\sigma^2
\).

Therefore,
\[
\mathrm{Var}(X)=\mathbb{E}[X^2]-\mathbb{E}[X]^2
=2\sigma^2-\sigma^2\frac{\pi}{2}=\frac{4-\pi}{2}\,\sigma^2.
\]

### Likelihood (i.i.d. sample) and MLE
For data \(x_1,\dots,x_n\ge 0\), the log-likelihood is
\[
\ell(\sigma)=\sum_{i=1}^n\log f(x_i;\sigma)
=\sum_{i=1}^n\log x_i - 2n\log\sigma - \frac{1}{2\sigma^2}\sum_{i=1}^n x_i^2.
\]

Differentiate and set to zero:
\[
\frac{d\ell}{d\sigma}=-\frac{2n}{\sigma}+\frac{1}{\sigma^3}\sum_{i=1}^n x_i^2=0
\quad\Longrightarrow\quad
\hat\sigma^2=\frac{1}{2n}\sum_{i=1}^n x_i^2.
\]

So the MLE is
\(
\hat\sigma=\sqrt{\frac{1}{2n}\sum_i x_i^2}.
\)


In [ ]:
def rayleigh_loglik(x, sigma):
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0 or np.any(x < 0):
        return -np.inf
    return np.sum(rayleigh_logpdf(x, sigma))


def rayleigh_mle_sigma(x):
    x = np.asarray(x, dtype=float)
    if np.any(x < 0):
        raise ValueError("Rayleigh data must be nonnegative")
    return np.sqrt(np.mean(x**2) / 2)


# MLE demo
sigma_true = 1.8
x = rayleigh_ppf(rng.random(5000), sigma=sigma_true)

sigma_hat = rayleigh_mle_sigma(x)

# Compare to SciPy's fit (fix loc=0 to match our parameterization)
loc_hat, scale_hat = stats.rayleigh.fit(x, floc=0)

print("sigma_true:", sigma_true)
print("sigma_hat (closed form):", sigma_hat)
print("sigma_hat (scipy fit):   ", scale_hat)
print("loc_hat (scipy fit):     ", loc_hat)


## 7) Sampling & simulation (NumPy-only)

### Inverse transform sampling
Because the CDF has a closed form,
\(
F(x)=1-e^{-x^2/(2\sigma^2)}
\),
we can sample by inversion.

Let \(U\sim\text{Uniform}(0,1)\). Set \(U=F(X)\) and solve:
\[
U = 1-e^{-X^2/(2\sigma^2)}
\quad\Longrightarrow\quad
X = \sigma\sqrt{-2\ln(1-U)}.
\]

In code, using `log1p` is numerically stable when \(U\) is close to 0:
\(
\ln(1-U)=\texttt{log1p(-U)}
\).

### Alternative sampler (geometric intuition)
Sample \(X,Y\overset{i.i.d.}{\sim}\mathcal{N}(0,\sigma^2)\) and return \(R=\sqrt{X^2+Y^2}\). This makes the 2D origin story explicit (but uses Gaussian sampling).


In [ ]:
def rayleigh_rvs_numpy(sigma, size, rng=None):
    """NumPy-only Rayleigh sampler via inverse CDF."""
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    if rng is None:
        rng = np.random.default_rng()

    u = rng.random(size)
    return rayleigh_ppf(u, sigma=sigma)


def rayleigh_rvs_via_normals(sigma, size, rng=None):
    """Sampler using the 2D Gaussian magnitude definition."""
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    if rng is None:
        rng = np.random.default_rng()

    if isinstance(size, tuple):
        shape = (2, *size)
    else:
        shape = (2, size)

    xy = rng.normal(loc=0.0, scale=sigma, size=shape)
    return np.sqrt(xy[0] ** 2 + xy[1] ** 2)


# Quick sampling sanity check
sigma = 1.3
s1 = rayleigh_rvs_numpy(sigma, size=200_000, rng=rng)
print("Mean (theory, sampler):", rayleigh_mean(sigma), s1.mean())


## 8) Visualization
We’ll look at:
- PDF vs Monte Carlo histogram
- CDF vs empirical CDF (ECDF)
- The exponential relationship for \(X^2\)


In [ ]:
def ecdf(samples):
    x = np.sort(np.asarray(samples))
    y = np.arange(1, x.size + 1) / x.size
    return x, y


sigma_viz = 1.6
n_viz = 120_000
samples = rayleigh_rvs_numpy(sigma_viz, size=n_viz, rng=rng)

# Histogram + PDF overlay
x_grid = np.linspace(0, np.quantile(samples, 0.999), 600)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=samples,
        nbinsx=80,
        histnorm="probability density",
        name="Monte Carlo",
        opacity=0.6,
    )
)
fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=rayleigh_pdf(x_grid, sigma_viz),
        mode="lines",
        name="theoretical pdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title=f"Rayleigh PDF vs Monte Carlo (σ={sigma_viz:g}, n={n_viz:,})",
    xaxis_title="x",
    yaxis_title="density",
)
fig.show()

# ECDF vs CDF
x_ecdf, y_ecdf = ecdf(samples)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_ecdf, y=y_ecdf, mode="lines", name="ECDF"))
fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=rayleigh_cdf(x_grid, sigma_viz),
        mode="lines",
        name="theoretical cdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title="CDF check: ECDF vs theoretical",
    xaxis_title="x",
    yaxis_title="cdf",
)
fig.show()

# Squaring property: X^2 ~ Exp(rate = 1/(2σ^2))
rate = 1 / (2 * sigma_viz**2)
y = samples**2
x_y = np.linspace(0, np.quantile(y, 0.999), 600)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=y,
        nbinsx=80,
        histnorm="probability density",
        name="Y = X² (MC)",
        opacity=0.6,
    )
)
fig.add_trace(
    go.Scatter(
        x=x_y,
        y=rate * np.exp(-rate * x_y),
        mode="lines",
        name=f"Exp(rate={rate:.3g}) pdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title="Squaring property: Y = X² is exponential",
    xaxis_title="y",
    yaxis_title="density",
)
fig.show()


## 9) SciPy integration (`scipy.stats.rayleigh`)

SciPy parameterizes Rayleigh as `stats.rayleigh(loc, scale)` with density
\(
\frac{x-\texttt{loc}}{\texttt{scale}^2}\exp\{-(x-\texttt{loc})^2/(2\texttt{scale}^2)\}
\) for \(x\ge\texttt{loc}\).

For the canonical distribution in this notebook, use `loc=0` and `scale=σ`.


In [ ]:
sigma = 1.4

dist = stats.rayleigh(loc=0, scale=sigma)

x = np.linspace(0, 8, 400)
pdf_scipy = dist.pdf(x)
cdf_scipy = dist.cdf(x)

# Compare SciPy vs our implementations
print("max |pdf diff|:", np.max(np.abs(pdf_scipy - rayleigh_pdf(x, sigma))))
print("max |cdf diff|:", np.max(np.abs(cdf_scipy - rayleigh_cdf(x, sigma))))

# Sampling
rvs = dist.rvs(size=5, random_state=rng)
print("rvs:", rvs)

# Fitting (MLE). Fix loc=0 for canonical Rayleigh.
true_sigma = 2.0
data = stats.rayleigh(scale=true_sigma).rvs(size=4000, random_state=rng)
loc_hat, scale_hat = stats.rayleigh.fit(data, floc=0)
print("true sigma:", true_sigma)
print("fit scale (sigma_hat):", scale_hat)


## 10) Statistical use cases

### A) Hypothesis testing
If \(X_i\sim\text{Rayleigh}(\sigma)\), then \((X_i/\sigma)^2\sim\chi^2_2\). Therefore
\[
\frac{1}{\sigma^2}\sum_{i=1}^n X_i^2 \sim \chi^2_{2n}.
\]
This gives exact tests and confidence intervals for \(\sigma\).

### B) Bayesian modeling
Using the squaring trick \(Y_i=X_i^2\), we have
\(Y_i\sim\text{Exp}(\text{rate}=\lambda)\) with \(\lambda=1/(2\sigma^2)\).
A Gamma prior on \(\lambda\) is conjugate, giving a simple posterior update.

### C) Generative modeling
In many signal-processing pipelines, you naturally model I/Q components as independent Gaussians. The amplitude \(\sqrt{I^2+Q^2}\) is Rayleigh (or Rice if there is a nonzero mean / line-of-sight component).


In [ ]:
# A) Exact test + confidence interval for sigma

sigma_true = 1.5
n = 400
x = stats.rayleigh(scale=sigma_true).rvs(size=n, random_state=rng)

sigma0 = 1.3  # null hypothesis

test_stat = np.sum(x**2) / sigma0**2
p_lower = stats.chi2.cdf(test_stat, df=2 * n)
p_upper = 1 - p_lower
p_two_sided = 2 * min(p_lower, p_upper)

print(f"H0: sigma = {sigma0:g}")
print("test statistic:", test_stat)
print("two-sided p-value:", p_two_sided)

# 95% CI for sigma via chi-square quantiles
alpha = 0.05
s = np.sum(x**2)
q_lo = stats.chi2.ppf(alpha / 2, df=2 * n)
q_hi = stats.chi2.ppf(1 - alpha / 2, df=2 * n)

sigma_ci_lo = np.sqrt(s / q_hi)
sigma_ci_hi = np.sqrt(s / q_lo)

print(f"95% CI for sigma: [{sigma_ci_lo:.4f}, {sigma_ci_hi:.4f}]")
print("true sigma:", sigma_true)


In [ ]:
# B) Bayesian update using Y = X^2 ~ Exp(rate = 1/(2 sigma^2))

sigma_true = 1.7
n = 200
x = stats.rayleigh(scale=sigma_true).rvs(size=n, random_state=rng)

y = x**2

# Prior on lambda = 1/(2 sigma^2): Gamma(shape=a0, rate=b0)
a0, b0 = 2.0, 1.0

# Exponential likelihood: p(y | lambda) = lambda * exp(-lambda y)
a_post = a0 + n
b_post = b0 + y.sum()

# Posterior samples of lambda, then transform to sigma
m = 50_000
lambda_samps = rng.gamma(shape=a_post, scale=1 / b_post, size=m)  # numpy uses scale=1/rate
sigma_samps = 1 / np.sqrt(2 * lambda_samps)

ci = np.quantile(sigma_samps, [0.025, 0.5, 0.975])

print("posterior median sigma:", ci[1])
print("95% credible interval:", (ci[0], ci[2]))
print("true sigma:", sigma_true)


In [ ]:
# C) Generative modeling: I/Q Gaussian -> Rayleigh amplitude

sigma = 1.2
n = 80_000

i = rng.normal(loc=0.0, scale=sigma, size=n)
q = rng.normal(loc=0.0, scale=sigma, size=n)
amp = np.sqrt(i**2 + q**2)

x_grid = np.linspace(0, np.quantile(amp, 0.999), 600)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=amp,
        nbinsx=80,
        histnorm="probability density",
        name="|I + jQ| (MC)",
        opacity=0.6,
    )
)
fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=rayleigh_pdf(x_grid, sigma),
        mode="lines",
        name="Rayleigh pdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title="Amplitude of complex Gaussian noise is Rayleigh",
    xaxis_title="amplitude",
    yaxis_title="density",
)
fig.show()


## 11) Pitfalls

- **Invalid parameters**: \(\sigma\le 0\) is not allowed; check inputs early.
- **Support mismatch**: Rayleigh is for \(x\ge 0\). Negative values often indicate a modeling/measurement issue (or that you should model a signed component instead).
- **SciPy `loc` parameter**: `stats.rayleigh(loc, scale)` shifts the support to \(x\ge\texttt{loc}\). If you want the canonical Rayleigh, fix `loc=0` when fitting.
- **Numerical stability**:
  - For the CDF near 0, use `expm1` (we did) to avoid subtracting nearly equal numbers.
  - For likelihoods, prefer `logpdf` to avoid underflow in the tail.
- **Model misspecification**: if there is a strong line-of-sight component, amplitude is often **Rice** (Rician), not Rayleigh.


## 12) Summary

- Rayleigh is a **continuous** distribution on \([0,\infty)\) with scale \(\sigma>0\).
- It is the **magnitude of a 2D zero-mean isotropic Gaussian** and a special case of **Weibull (shape 2)**.
- Closed forms for PDF/CDF/quantiles make it easy to **simulate** (inverse CDF) and **fit** (closed-form MLE).
- \(X^2\) being **exponential** is a powerful trick for derivations and Bayesian modeling.

### References
- SciPy documentation: `scipy.stats.rayleigh`
- Wikipedia: Rayleigh distribution
- For the nonzero-mean generalization: Rice (Rician) distribution
